In [4]:
!python3.11 -m pip install seaborn


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [5]:
import pandas as pd
import seaborn as sns
from functools import partial
import numpy as np


def getP(row, eps) :
    r = row["r"]
    return row[f"P_{r}_{eps}"]

def prob_unif_data(data, eps, pnscol):
    a = data[pnscol].apply(min)
    b = data[pnscol].apply(max)
    n = data[pnscol].apply(len)
    L = b - a
    
    num = (1 + (1 + 2*eps)**(2 - n) - 2 * (1+ eps) ** (2-n)) 
    den = (1 - L ** (n - 2) - (n - 2) * (1 - L) * L ** (n - 2))
    return num/den


def load(file, eps,llratio, pnscol, duplicate, fix, maxR):
    dt = pd.read_pickle(file).copy()
    #dt = dt[np.abs(dt.pns_l_emcc - dt.pns_u_emcc) > 0.02].copy()
    dt["r"] = dt[pnscol].apply(len)
    dt['duplicate'] = duplicate
    dt["fix"] = fix
    dt["maxR"] = maxR
    dt["pnss"] = dt[pnscol].apply(lambda pns: pns.astype(float))
    dt["P"] = dt.apply(partial(getP, eps=eps), axis=1)
    dt["Punif"]=prob_unif_data(dt, eps, pnscol)
    dt["eps"] = str(eps)
    return dt[["modelFile", "r", "P", "duplicate", "fix", "pnss", "Punif", pnscol, "pns_l_emcc", "pns_u_emcc", "pns_l_ccve", "pns_u_ccve", "eps", "maxR"]]
    

ModuleNotFoundError: No module named 'seaborn'

# exploration ($r_{max}$)

In [ ]:
data = pd.DataFrame([
    [True, True, 0.01, 0.98, "pns098", "s1d", "P_kl_1e8_s1d_100iter_fix098_diplicate_0.01.pickle"],
    [True, True, 0.05, 0.98,"pns098", "s1d", "P_kl_1e8_s1d_100iter_fix098_diplicate_0.05.pickle"],
    [True, True, 0.1, 0.98,"pns098",  "s1d", "P_kl_1e8_s1d_100iter_fix098_diplicate_0.1.pickle"],
    [False, True, 0.01, 0.98,"pns098", "s1b", "P_kl_1e8_s1d_100iter_fix098_0.01.pickle"],
    [False, True, 0.05, 0.98,"pns098", "s1b", "P_kl_1e8_s1d_100iter_fix098_0.05.pickle"],
    [False, True, 0.1, 0.98,"pns098", "s1b", "P_kl_1e8_s1d_100iter_fix098_0.1.pickle"],
], columns=['duplicate', "fix", "eps", "llratio","pnsCol", "dataset", "file"])

data = pd.DataFrame([
    #[True, True, 0.1, 0.98, "pns", "s1d", "probabs/P50_0.1.pickle", "50"],
    #[True, True, 0.1, 0.98, "pns", "s1d", "probabs/P100_0.1.pickle", "100"],
    #[True, True, 0.1, 0.98, "pns", "s1d", "probabs/P150_0.1.pickle", "150"],
    [True, True, 0.1, 0.98, "pns", "s1d", "probabs/P200_0.1.pickle", "200"],
], columns=['duplicate', "fix", "eps", "llratio","pnsCol", "dataset", "file", "maxR"])



#d1 = load("P_kl_1e8_s1d_100iter_fix098_0.01.pickle", 0.01)
#d2 = load("P_kl_1e8_s1d_100iter_fix098_0.05.pickle", 0.05)
#d3 = load("P_kl_1e8_s1d_100iter_fix098_0.1.pickle", 0.1)

dt = pd.concat((
    load(d.file, d.eps, d.llratio, d.pnsCol, d.duplicate, d.fix, d.maxR).reset_index()
    for index, d in data.iterrows())).reset_index()

#dt = d1.merge(d2[["modelFile", "P0.05"]], on="modelFile").merge(d3[["modelFile", "P0.1"]], on="modelFile")

In [ ]:
dt["a"] = dt.pnss.apply(min)
dt["b"] = dt.pnss.apply(max)
dt["L"] = dt.pnss.apply(max) - dt.pnss.apply(min)

In [ ]:
pd.__version__

In [ ]:
dt.to_json("P_0.1_200.json")

In [ ]:
pd.read_json("P_0.1_200.json").pnss.apply(min)

In [ ]:
import convergence
from functools import partial
f = partial(convergence.fit_beta, eps=0.1, precision=0.000000001)

x = dt[dt["P"] > 1].copy()

ab = x.pns.apply(partial(np.array, dtype=float)).apply(f)

In [ ]:
x[x["a"] > 0.01].iloc[1].pns

In [ ]:
x.shape

In [ ]:
from ipywidgets import interact
import ipywidgets

@interact(i=ipywidgets.IntSlider(10, min=0, max=x.shape[0]-1))
def plt(i):
    sns.histplot(x.pns.iloc[i], bins=20)
    print(x.iloc[i].P, x.iloc[i].pns.min())

In [ ]:
import convergence
row = 109
eps = 0.1
dta = x.iloc[row].pns.astype(float)
@interact(i=ipywidgets.IntSlider(5, min=5, max=len(dta)))
def plt(i):
    print(convergence.p_eps_convergence(dta[:i], eps))

In [ ]:
upper = 140
row = 109 #53

def print_case(data, k, eps):
    pns = data[:k].astype(float)
    alpha, beta = convergence.fit_beta(pns, eps, 0.0000001)
    l = max(pns) - min(pns)
    delta = 2*eps*l

    element = convergence._pjoint(0, delta/2, l, alpha, beta, k)
    print(element, min(pns), max(pns), l, convergence.p_eps_convergence(pns, eps)[0], alpha, beta)



print_case(x.iloc[row].pns, 140, 0.1)
print_case(x.iloc[row].pns, 200, 0.1)


x.iloc[row].pns.shape

In [ ]:
a = ab.apply(lambda a:a[0]).values
b = ab.apply(lambda a:a[1]).values
sns.scatterplot(a,b)

In [ ]:

dt1 = dt[(dt.eps == "0.1") & (dt.r == 200)]
print(dt1.shape)
dt2 = dt1[(dt.P > 1.0)]

In [ ]:
dt2.iloc[dt2.P.argmax()]

In [ ]:
from scipy.stats.distributions import beta 
x = np.linspace(beta.ppf(0.01, a, b), beta.ppf(0.99, a, b), 100)


In [ ]:
sns.lineplot(x, beta.pdf(x, a, b))

In [ ]:
dt = dt[dt.P <= 1.0]
dt["eps dup"] = dt["eps"].astype(str) + " " + dt["duplicate"].astype(str)

In [ ]:
sns.set_theme(style="white", palette="tab10")
sns.displot(dt[dt.eps == "0.1"], x="P", hue="duplicate", kind="ecdf")
sns.displot(dt, x="Punif", hue="eps", kind="kde", fill=True)

# S1B duplicate exploration

In [ ]:
d1d = load("P_kl_1e8_s1b_100iter_duplicated_0.01.pickle", 0.01)
d3d = load("P_kl_1e8_s1b_100iter_duplicated_0.1.pickle", 0.1)
dtd = pd.concat((d1d,d3d)).reset_index()
d3d['dup'] = True

In [ ]:
sns.set_theme(style="white", palette="tab10")
sns.displot(dtd, x="P", hue="eps",   fill=True)
sns.displot(dtd, x="Punif", hue="eps", fill=True)

In [ ]:
d1['data dup']=False
d1d['data dup']=True
sns.displot(pd.concat((d1, d1d)).reset_index(), x="P", hue="data dup")


# S1D exploration (no dup)

In [ ]:
f1 = load("P_kl_1e8_s1d_100iter_0.01.pickle", 0.01)
f2 = load("P_kl_1e8_s1d_100iter_0.05.pickle", 0.05)
f3 = load("P_kl_1e8_s1d_100iter_0.1.pickle", 0.1)

ft = pd.concat((f1,f2,f3)).reset_index()
#dt = d1.merge(d2[["modelFile", "P0.05"]], on="modelFile").merge(d3[["modelFile", "P0.1"]], on="modelFile")

In [ ]:
sns.set_theme(style="white", palette="tab10")
sns.displot(ft, x="P", hue="eps", kind="ecdf")
sns.displot(ft, x="Punif", hue="eps", kind="kde", fill=True)

In [ ]:
sns.scatterplot(y = f1.P, x = f1.r)

In [ ]:
pltd = (d1, d1d, f1)
#pltd = (d3, d3d, f3)
pltd[0]['data set']="S1B"
pltd[1]['data set']="S1B_DUP"
pltd[2]['data set']="S1D"
sns.displot(pd.concat(pltd).reset_index(), x="P", kind="ecdf", hue="data set")


In [ ]:
s = pd.read_pickle("pns_kl_1e8_s1d.pickle")
s = dt[(dt.eps=="0.1") & ( ~ dt.duplicate)].merge(s, on="modelFile")

In [ ]:
sns.scatterplot( s.pnss.apply(min),s.P)

# s1d 200 exploration

In [ ]:
d_50 = pd.read_pickle("P50_0.1.pickle")
d_100 = pd.read_pickle("P100_0.1.pickle")
d_150 = pd.read_pickle("P150_0.1.pickle")
d_200 = pd.read_pickle("P200_0.1.pickle")


In [ ]:
d_50["P_199_0.1"]


In [ ]:
d_50.pns098.apply(len)